In [75]:
import pandas as pd
import glob
import os
import numpy as np
from time import time
import logging
import gensim
import bz2
import PyPDF2
import requests
import json

In [97]:
dataset = []

path = '../datas/scrap/person/'
allFiles = glob.glob(os.path.join(path, '*.csv'))

# Load and concactenate all the files into one dataset.
for file_ in allFiles:
    data = pd.read_csv(file_)
    dataset += [data]
datas = pd.concat(dataset)

print(len(data.columns))  
#print('Length of the dataset', len(dataset))

22


In [98]:
datas[datas["FirstName"] == "Roger"]

,Unnamed: 0,DateOfBirth,DateOfDeath,FirstName,GenderAsString,ID,Language,LastName,MaritalStatus,MaritalStatusText,...,Modified,NativeLanguage,NumberOfChildren,OfficialName,PersonIdCode,PersonNumber,PlaceOfBirthCanton,PlaceOfBirthCity,Title,TitleText
564,564,1917-12-11T00:00:00,1998-02-13T00:00:00,Roger,m,641,FR,Schaffter,NaN,NaN,...,2015-05-17T21:18:19.387,NaN,NaN,NaN,1090.0,641,NaN,NaN,NaN,NaN
566,566,1914-12-02T00:00:00,1996-03-26T00:00:00,Roger,m,643,FR,Dafflon,NaN,NaN,...,2015-05-17T21:18:19.387,NaN,NaN,NaN,1092.0,643,NaN,NaN,NaN,NaN
801,801,1928-07-17T00:00:00,2005-06-29T00:00:00,Roger,m,1179,FR,Besuchet,NaN,NaN,...,2015-05-17T21:18:19.387,F,NaN,Besuchet,NaN,1179,NaN,NaN,NaN,NaN
843,843,1921-12-26T00:00:00,2008-11-18T00:00:00,Roger,m,1223,FR,Mugny,NaN,NaN,...,2015-05-17T21:18:19.387,F,NaN,Mugny Roger,NaN,1223,NaN,NaN,NaN,NaN
880,880,1973-03-23T00:00:00,NaN,Roger,m,1279,FR,Nordmann,2.0,marié(e),...,2015-11-13T14:49:35.527,F,2.0,Nordmann,2648.0,1279,Vaud,Lausanne,134.0,lic. ès sc. pol.


In [77]:
datas = datas.dropna(axis = 0, subset = ["PersonIdCode"])

In [78]:

ids = datas.PersonIdCode.unique().astype(int)


In [43]:
url_portrait = "https://www.parlament.ch/sitecollectionimages/profil/portrait-260/"
for id in ids:
    
    f = open('../datas/pictures/' + str(id) + '.jpg','wb')
    f.write(requests.get(url_portrait + str(id) + '.jpg').content)
    f.close()

KeyboardInterrupt: 

In [96]:
datas[datas["FirstName"] == "Roger"]

,Unnamed: 0,DateOfBirth,DateOfDeath,FirstName,GenderAsString,ID,Language,LastName,MaritalStatus,MaritalStatusText,...,NativeLanguage,NumberOfChildren,OfficialName,PersonIdCode,PersonNumber,PlaceOfBirthCanton,PlaceOfBirthCity,Title,TitleText,id
564,564,1917-12-11T00:00:00,1998-02-13T00:00:00,Roger,m,641,FR,Schaffter,NaN,NaN,...,NaN,NaN,NaN,1090.0,641,NaN,NaN,NaN,NaN,Roger Schaffter
566,566,1914-12-02T00:00:00,1996-03-26T00:00:00,Roger,m,643,FR,Dafflon,NaN,NaN,...,NaN,NaN,NaN,1092.0,643,NaN,NaN,NaN,NaN,Roger Dafflon
880,880,1973-03-23T00:00:00,NaN,Roger,m,1279,FR,Nordmann,2.0,marié(e),...,F,2.0,Nordmann,2648.0,1279,Vaud,Lausanne,134.0,lic. ès sc. pol.,Roger Nordmann


In [80]:
datas['id'] = datas['FirstName']+' '+datas['LastName']
links = datas[["PersonIdCode"  ,"id"]]

In [90]:
dictionary = dict(zip(datas["id"].tolist(),datas["PersonIdCode"].astype(int).values.tolist()))

In [91]:
dictionary

{'Achille Casanova': 2581,
 'Adalbert Durrer': 2378,
 'Adolf Ogi': 2132,
 'Adrian Amstutz': 2584,
 'Adriano Cavadini': 2033,
 'Adriano Imfeld': 2571,
 'Agnes Weber': 2429,
 'Alain Berset': 2642,
 'Alain Schweingruber': 2344,
 'Albert Eggli': 1019,
 'Albert Rüttimann': 2153,
 'Alberto Stefani': 1159,
 'Albrecht Rychen': 2154,
 'Alex Heim': 2452,
 'Alex Kuprecht': 2641,
 'Alexander Euler': 2056,
 'Alexander Tschäppät': 2328,
 'Alexis Gobet': 2293,
 'Alfons Müller': 1132,
 'Alfred Affolter': 1097,
 'Alfred Neukomm': 2127,
 'Alma Bacciarini': 1101,
 'Alois Dobler': 2209,
 'Amélia Christinat': 1012,
 'Andrea Hämmerle': 2298,
 'Andreas Dürr': 1110,
 'Andreas Gerwig': 1116,
 'Andreas Gross': 2296,
 'Andreas Herczog': 2083,
 'Andreas Iten': 2218,
 'Andreas Zeller': 2658,
 'André Bugnon': 2477,
 'André Daguet': 2591,
 'André Gautier': 2213,
 'André Perey': 2135,
 'André Reymond': 2626,
 'Angeline Fankhauser': 2058,
 'Anita Fetz': 2061,
 'Anita Thanei': 2423,
 'Anne-Catherine Menétrey-Savary': 2

In [25]:
links.to_json("linkspersons.json")

In [92]:
with open('linkspersons.json', 'w') as outfile:
    json.dump(dictionary, outfile)